# Demo of two-motor dynamic limit signal

## Overview

Some instrument designs need protection against accidental collision between moving parts during routine operations.  In such cases, each of the axes may be operating between its valid range but an interim (in-motion) state can pose the possibility of a collision.

One such possible collision is when arms of a diffractometer (such as $\theta$ and $2\theta$, known here as `theta` and `ttheta`, respectively) collide causing damage to beam transport apparatus and consequential instrumental downtime.

To prevent the collision *in this case*, the $2\theta$ axis must be at least $\delta$ degrees above the $\theta$ axis.  Empirically, $\delta$ of 3 degrees is sufficient protection.

From a controls safety view, we provide an EPICS PV calculation that is zero when the move is not permitted and one when: ($2\theta - \theta) \ge \delta$.  We'll monitor that PV in Bluesky to add a [suspender](https://blueskyproject.io/bluesky/state-machine.html#automated-suspension) that can interrupt the scan (via the [`bluesky.RunEngine`](https://blueskyproject.io/bluesky/run_engine_api.html?highlight=runengine)) if the permit is removed.  When the RunEngine handles an interruption involving [*movable devices*](https://blueskyproject.io/bluesky/hardware.html?highlight=movables#settable-movable-device), it sends a stop to each of the movables involved.  Thus, when the dynamic limit permit is removed, both motors are stopped and the scan pauses, waiting for external interaction to clear the condition.

<details>
[Here](https://github.com/bluesky/bluesky/blob/4fab894bddbd4a563f28852ea3171b87140ae7b9/bluesky/run_engine.py#L1034-L1036) is where bluesky tells the motors to stop:

```
            if justification is not None:
                print("Justification for this suspension:\n%s" % justification)
            for current_run in self._run_bundlers.values():
                current_run.record_interruption('resume')
            # During suspend, all motors should be stopped. Call stop() on
            # every object we ever set().
            self._stop_movable_objects(success=True)
```

If the RunEngine is started while the dynamic limit permit calculation is zero, the RunEngine will pause immediately.  Here is an example:

```
In [21]: uid = RE(th_tth_scan([noisy, th_tth_permit], 8, 6, points=4, min_sep=3))                               
At least one suspender has tripped. The plan will begin when all suspenders are ready. Justification:
    1. Signal th_tth_permit is low

Suspending... To get to the prompt, hit Ctrl-C twice to pause.
```

We might need to write our own suspender if none of the [provided suspenders](https://blueskyproject.io/bluesky/state-machine.html#built-in-suspenders) will do the job we want.  There is [an example](https://github.com/BCDA-APS/apstools/blob/master/apstools/suspenders.py#L25) of a custom suspender in [apstools](https://apstools.readthedocs.io).

</details>

## Summary

Any time the motors are moved by the bluesky RunEngine, they will be stopped if the dynamic limit permit calculation goes to zero and the scan will pause.


## EPICS setup

We start with two motor axes defined in EPICS.  Here, we run the docker image [`prjemian/synApps-6.1`](https://hub.docker.com/r/prjemian/synapps-6.1) to make the [EPICS IOC simulator run in a docker container](https://github.com/prjemian/epics-docker/tree/master/n3_synApps#use-a-custom-ioc-prefix) with IOC prefix `sky:`.  These are motor PVs: `sky:m1` and `sky:m2` as shown.

![EPICS motor GUI screens](resources/demo_dynamic_limits_motors.png)

To compute a dynamic limit between the two motor axes, we use a *userCalc* (EPICS [swait]() record), `sky:userCalc1` with settings as shown.

1. Set the description field to describe what this does.
1. Monitor each motor's readback (`.RBV`) value.  The readback value is the motor record's *best* knowledge of the actual motor position.
1. `sky:userCalc1.INAN` = `sky:m1.RBV`, the value will be in `A` once the motor moves.
1. `sky:userCalc1.INBN` = `sky:m2.RBV`, the value will be in `B` once the motor moves.
1. Change the calculation's `.SCAN` field from *Passive* (calculates only when requested) to *I/O Intr* (calculate when any input changes, based on each field's *TRIGGER?* setting).
1. Enter the angle of minimum approach (3) into the `sky:userCalc1.C` field.  This will be the PV to change this number.
1. Enter the permit calculation: `(B-A)>=C`

The calculated result (in `sky:userCalc1.VAL`, `sky:userCalc1` for short) once either of the motors move.

![motion permit calculation](resources/demo_dynamic_limits_permit_calc.png)

To scan, we want a "detector".  Let's use another *userCalc* (`sky:userCalc2`) to simulate a noisy detector with a random number generator.  We'll update this detector only when either motor moves (same as with the permit calculation) setting its `.SCAN` to *I/O Intr*.  The setup is shown in the next screen view image:

![noisy detector simulation](resources/demo_dynamic_limits_permit_noisy.png)

## Bluesky setup

In [1]:
# we'll need these imports
from instrument.collection import *
from bluesky import plans as bp
from bluesky import plan_stubs as bps
from ophyd import Component, Device, EpicsMotor, EpicsSignal

I Wed-14:39:46 - ############################################################ startup
I Wed-14:39:46 - logging started
I Wed-14:39:46 - logging level = 10
I Wed-14:39:46 - /home/prjemian/Documents/projects/use_bluesky/lessons/instrument/collection.py
I Wed-14:39:46 - /home/prjemian/Documents/projects/use_bluesky/lessons/instrument/mpl/notebook.py


Activating auto-logging. Current session state plus future input saved.
Filename       : /home/prjemian/Documents/projects/use_bluesky/lessons/.logs/ipython_console.log
Mode           : rotate
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


I Wed-14:39:46 - bluesky framework
I Wed-14:39:46 - /home/prjemian/Documents/projects/use_bluesky/lessons/instrument/framework/check_python.py
I Wed-14:39:46 - /home/prjemian/Documents/projects/use_bluesky/lessons/instrument/framework/check_bluesky.py
I Wed-14:39:47 - /home/prjemian/Documents/projects/use_bluesky/lessons/instrument/framework/initialize.py
I Wed-14:39:48 - /home/prjemian/Documents/projects/use_bluesky/lessons/instrument/framework/metadata.py
I Wed-14:39:48 - /home/prjemian/Documents/projects/use_bluesky/lessons/instrument/framework/callbacks.py
I Wed-14:39:48 - writing to SPEC file: /home/prjemian/Documents/projects/use_bluesky/lessons/20200902-143948.dat
I Wed-14:39:48 -    >>>>   Using default SPEC file name   <<<<
I Wed-14:39:48 -    file will be created when bluesky ends its next scan
I Wed-14:39:48 -    to change SPEC file, use command:   newSpecFile('title')


The active progress bar is great to indicate an activity in-progress but it interefers with the display in the jupyter notebook.  Disable the progress bar

In [2]:
RE.waiting_hook = None

symbol | PV (here) | meaning
--- | --- | ---
`theta` | sky:m1 | (`th`) EPICS motor record
`ttheta` | sky:m2 | (`tth`) EPICS motor record
`th_tth_permit` | sky:userCalc1.VAL | result of EPICS calculation (swait record: 1=permit, 0=not), updates when either motor moves
`th_tth_min` | sky:userCalc1.C | minimum permitted `tth-th`
`noisy` | sky:userCalc2.VAL | detector (random number generator) - integer, updates when either motor moves
`noisy_scale` | sky:userCalc2.C | scale factor for `noisy`

In [3]:
noisy = EpicsSignal("sky:userCalc2.VAL", name="noisy")
noisy_scale = EpicsSignal("sky:userCalc2.C", name="noisy_scale")
th_tth_min = EpicsSignal("sky:userCalc1.C", name="th_tth_min")
th_tth_permit = EpicsSignal("sky:userCalc1.VAL", name="th_tth_permit")
theta = EpicsMotor("sky:m1", name="theta", labels=["motors",])
ttheta = EpicsMotor("sky:m2", name="ttheta", labels=["motors",])

In [4]:
# for this demo, we want to reset the ttheta motor backlash to the default condition
# when the IOC is first created.  We'll change this later in the demo.
# We drop to PyEpics since backlash is not part of `ophyd.EpicsMotor`
import epics
tth_pv = ttheta.prefix
epics.caget(f"{tth_pv}.BDST", 0)
epics.caget(f"{tth_pv}.BVEL", 0)
del tth_pv
# We do this ONLY in the simulator.  Real instrument motors may need different settings!

## Custom Bluesky plan for $\theta:2\theta$ scan

Define a plan for a coupled theta:2theta scan.

In [5]:
def th_tth_scan(detectors, tth_start, tth_end, points=11, min_sep=None):
    """
    run a coupled theta:2theta scan
    """
    min_sep = abs(min_sep or 3)
    old_sep = th_tth_min.get()

    # check end points first!
    if abs(tth_start/2) < min_sep:
        print(
            "Starting point below allowed minimum:"
            f" |{tth_start/2:.4f}| < |{min_sep:.4f}|")
        return
    if abs(tth_end/2) < min_sep:
        print(
            "Ending point below allowed minimum:"
            f" |{tth_end/2:.4f}| < |{min_sep:.4f}|")
        return

    yield from bps.mv(th_tth_min, min_sep)
    yield from bp.scan(
        detectors,
        theta, tth_start/2, tth_end/2,
        ttheta, tth_start, tth_end,
        points
    )

    # reset the previous minimum
    yield from bps.mv(th_tth_min, old_sep)

Try a scan that we know will fail the test for minimum separation.

In [6]:
RE(th_tth_scan([noisy, th_tth_permit], 5, 25, points=11, min_sep=3))

Starting point below allowed minimum: |2.5000| < |3.0000|


()

Swap the two end points, that also fails.

In [7]:
RE(th_tth_scan([noisy, th_tth_permit], 25, 5, points=11, min_sep=3))

Ending point below allowed minimum: |2.5000| < |3.0000|


()

This scan is successful.

In [8]:
RE(th_tth_scan([noisy, th_tth_permit], 9, 6, points=11, min_sep=3))



Transient Scan ID: 1     Time: 2020-09-02 14:39:49
Persistent Unique Scan ID: '7e4bd027-efb8-43b0-b457-3c9ccdf7cc6b'
New stream: 'primary'


<IPython.core.display.Javascript object>

+-----------+------------+------------+------------+------------+---------------+
|   seq_num |       time |      theta |     ttheta |      noisy | th_tth_permit |
+-----------+------------+------------+------------+------------+---------------+
|         1 | 14:39:54.6 |    4.50000 |    9.00000 |      34490 |       1.00000 |
|         2 | 14:39:58.5 |    4.35000 |    8.70000 |      72792 |       1.00000 |
|         3 | 14:40:02.5 |    4.20000 |    8.40000 |      70828 |       1.00000 |
|         4 | 14:40:06.6 |    4.05000 |    8.10000 |      70713 |       1.00000 |
|         5 | 14:40:10.6 |    3.90000 |    7.80000 |      40636 |       1.00000 |
|         6 | 14:40:14.6 |    3.75000 |    7.50000 |      17438 |       1.00000 |
|         7 | 14:40:18.6 |    3.60000 |    7.20000 |      91379 |       1.00000 |
|         8 | 14:40:22.6 |    3.45000 |    6.90000 |      14334 |       1.00000 |
|         9 | 14:40:26.6 |    3.30000 |    6.60000 |      87701 |       1.00000 |
|        10 | 14

('7e4bd027-efb8-43b0-b457-3c9ccdf7cc6b',)

So far, have not registered a permit denied.  Have not encountered a condition where permit _would_ be denied.

## Try to provoke a permit denied

Check the backlash parameters for 2theta motor.  Since they are not part of `ophyd.EpicsMotor`, need to make a custom motor.

In [9]:
class MyMotor(EpicsMotor):
    backlash = Component(EpicsSignal, ".BDST")
    backlash_velocity = Component(EpicsSignal, ".BVEL")

# remake the two motors with backlash support
theta = MyMotor("sky:m1", name="theta", labels=["motors",])
ttheta = MyMotor("sky:m2", name="ttheta", labels=["motors",])

print(f"theta backlash distance: {theta.backlash.get()} ({theta.motor_egu.get()})")
print(f"theta backlash velocity: {theta.backlash_velocity.get()} ({theta.motor_egu.get()}/s)")

print(f"ttheta backlash distance: {ttheta.backlash.get()} ({ttheta.motor_egu.get()})")
print(f"ttheta backlash velocity: {ttheta.backlash_velocity.get()} ({ttheta.motor_egu.get()}/s)")

theta backlash distance: 0.0 (degrees)
theta backlash velocity: 1.0 (degrees/s)
ttheta backlash distance: 0.5 (degrees)
ttheta backlash velocity: 0.2 (degrees/s)


Reduce (just) the backlash velocity and set a backlash distance for `ttheta`.

In [10]:
%mov ttheta.backlash 0.5
%mov ttheta.backlash_velocity 0.2
print(f"ttheta backlash distance: {ttheta.backlash.get()} ({ttheta.motor_egu.get()})")
print(f"ttheta backlash velocity: {ttheta.backlash_velocity.get()} ({ttheta.motor_egu.get()}/s)")

ttheta backlash distance: 0.5 (degrees)
ttheta backlash velocity: 0.2 (degrees/s)


Scan again over a shorter range.

In [11]:
RE(th_tth_scan([noisy, th_tth_permit], 7, 6, points=4, min_sep=3))



Transient Scan ID: 2     Time: 2020-09-02 14:40:35
Persistent Unique Scan ID: '0bd68f55-321d-4834-b607-262d671d83e8'
New stream: 'primary'
+-----------+------------+------------+------------+------------+---------------+
|   seq_num |       time |      theta |     ttheta |      noisy | th_tth_permit |
+-----------+------------+------------+------------+------------+---------------+
|         1 | 14:40:38.8 |    3.50000 |    7.00000 |      43796 |       1.00000 |
|         2 | 14:40:42.8 |    3.33000 |    6.67000 |      86621 |       1.00000 |
|         3 | 14:40:46.8 |    3.17000 |    6.33000 |      46017 |       1.00000 |
|         4 | 14:40:50.8 |    3.00000 |    6.00000 |      71112 |       1.00000 |
+-----------+------------+------------+------------+------------+---------------+
generator scan ['0bd68f55'] (scan num: 2)


('0bd68f55-321d-4834-b607-262d671d83e8',)

**Still**, the permit signal never shows a zero value because it is recorded at the **end** of each step.

Let's monitor the signal _during_ the scan.

In [12]:
sd.monitors.append(th_tth_permit)

Repeat the scan, collecting the new info.  We'll inspect the monitors after the scan is done.  (The RunEngine returns a list of all the run uids created by the plan.  Capture this list.)

In [13]:
uids = RE(th_tth_scan([noisy, th_tth_permit], 8, 6, points=4, min_sep=3))



Transient Scan ID: 3     Time: 2020-09-02 14:40:51
Persistent Unique Scan ID: 'b0634792-0f0e-45b7-81fd-8c3cda50cd92'
New stream: 'th_tth_permit_monitor'


<IPython.core.display.Javascript object>

New stream: 'primary'
+-----------+------------+------------+------------+------------+---------------+
|   seq_num |       time |      theta |     ttheta |      noisy | th_tth_permit |
+-----------+------------+------------+------------+------------+---------------+
|         1 | 14:40:56.0 |    4.00000 |    8.00000 |      96612 |       1.00000 |
|         2 | 14:41:00.3 |    3.67000 |    7.33000 |      11617 |       1.00000 |
|         3 | 14:41:04.6 |    3.33000 |    6.67000 |      61225 |       1.00000 |
|         4 | 14:41:08.9 |    3.00000 |    6.00000 |      81115 |       1.00000 |
+-----------+------------+------------+------------+------------+---------------+
generator scan ['b0634792'] (scan num: 3)


Aha!  The first plot shows the monitored values.  It is clear the signal **does** go to 0 and then come back to 1.

Look at the monitored data.  First get the run object from databroker (indexed by the first uid in the list).  From the run object, return the monitored data in a table.  Python prints this object if it is not assigned.  All this in one step.

In [14]:
db[uids[0]].table("th_tth_permit_monitor")

,time,th_tth_permit
seq_num,,
2,2020-09-02 19:40:51.300086498,1.0
1,2020-09-02 19:40:51.299906492,1.0
3,2020-09-02 19:40:51.352354050,1.0
4,2020-09-02 19:40:51.356396437,1.0
5,2020-09-02 19:40:51.450339556,1.0
6,2020-09-02 19:40:51.453392744,1.0
7,2020-09-02 19:40:51.550068140,1.0
8,2020-09-02 19:40:51.552318096,1.0
9,2020-09-02 19:40:51.650589228,0.0


We see some zero values (and then return to one) indicating occasional removal of the dynamic limit calculation permit.  Since we only saw these when we added a backlash correction, we understand these dynamic violations of the limits are exactly what we hoped to intercept.

## Create a suspender

Block the RunEngine when the permit fails.

N.B. Might need to consider the special case where the permit fails when first starting the run.  Why did that fail? (We'll answer that soon.)

Following this example: https://blueskyproject.io/bluesky/state-machine.html#example-suspend-a-plan-if-the-beam-current-dips-low,
we'll do similar but know that our signal is a boolean, that indicates *no permit* when low.  Suspends interrupt the `RE` as long as the signal is invalid and automatically resume if the signal becomes valid again.  Let's see how this works here.

In [15]:
from bluesky.suspenders import SuspendBoolLow
sus = SuspendBoolLow(th_tth_permit)
RE.install_suspender(sus)

Repeat the scan.

In [16]:
uids = RE(th_tth_scan([noisy, th_tth_permit], 8, 6, points=4, min_sep=3))



Transient Scan ID: 4     Time: 2020-09-02 14:41:09
Persistent Unique Scan ID: '5283e9d8-7660-4240-9ebc-02ad36254110'
New stream: 'th_tth_permit_monitor'
Suspending....To get prompt hit Ctrl-C twice to pause.
Suspension occurred at 2020-09-02 14:41:09.
Suspender SuspendBoolLow(EpicsSignal(read_pv='sky:userCalc1.VAL', name='th_tth_permit', value=1.0, timestamp=1599075669.671728, tolerance=1e-05, auto_monitor=False, string=False, write_pv='sky:userCalc1.VAL', limits=False, put_complete=False), sleep=0, pre_plan=None, post_plan=None,tripped_message=) reports a return to nominal conditions. Will sleep for 0 seconds and then release suspension at 2020-09-02 14:41:09.
Justification for this suspension:
Signal th_tth_permit is low


Run aborted
Traceback (most recent call last):
  File "/home/prjemian/Apps/anaconda/envs/bluesky_2020_9/lib/python3.8/site-packages/bluesky/run_engine.py", line 1365, in _run
    msg = self._plan_stack[-1].send(resp)
  File "/home/prjemian/Apps/anaconda/envs/bluesky_2020_9/lib/python3.8/site-packages/bluesky/preprocessors.py", line 1307, in __call__
    return (yield from plan)
  File "/home/prjemian/Apps/anaconda/envs/bluesky_2020_9/lib/python3.8/site-packages/bluesky/preprocessors.py", line 1160, in baseline_wrapper
    return (yield from plan)
  File "/home/prjemian/Apps/anaconda/envs/bluesky_2020_9/lib/python3.8/site-packages/bluesky/preprocessors.py", line 803, in monitor_during_wrapper
    return (yield from plan2)
  File "/home/prjemian/Apps/anaconda/envs/bluesky_2020_9/lib/python3.8/site-packages/bluesky/preprocessors.py", line 170, in plan_mutator
    raise ex
  File "/home/prjemian/Apps/anaconda/envs/bluesky_2020_9/lib/python3.8/site-packages/bluesky/preprocessors.py", line 1

FailedStatus: MoveStatus(done=True, pos=theta, elapsed=0.1, success=False, settle_time=0.0)

**Failed** before the scan started, when moving the motors to the start position, with th=3.05 and tth=6.05.  We saw this in the previous scans, as well.  This is suspiciously close to the minimum separation and is likely because we moved `theta` *before* moving `ttheta`. 

The signal does not automatically clear since it is only computed when the motor readback value changes.  We can clear this by moving the `ttheta` motor manually.  If we change our plan logic to move `ttheta` first, we might avoid this altogether.  Yet, we could encounter the problem, depending on where the motors are before the scan or the direction of scanning.

Motion is not permitted due to our computation of dynamic limits.  We must move the motors so that the limit permit is restored before we can scan.  We can move the motor from the command line. (This works since this does not use the RunEngine so the suspender is not checked.)

In [17]:
%mov ttheta 6.1

Suspender SuspendBoolLow(EpicsSignal(read_pv='sky:userCalc1.VAL', name='th_tth_permit', value=1.0, timestamp=1599075692.091107, tolerance=1e-05, auto_monitor=False, string=False, write_pv='sky:userCalc1.VAL', limits=False, put_complete=False), sleep=0, pre_plan=None, post_plan=None,tripped_message=) reports a return to nominal conditions. Will sleep for 0 seconds and then release suspension at 2020-09-02 14:41:32.
ttheta:  20%|█████                    | 0.01/0.05 [00:00<00:00, 17.73s/degrees]
ttheta:  40%|██████████               | 0.02/0.05 [00:00<00:00, 13.87s/degrees]
ttheta:  60%|███████████████          | 0.03/0.05 [00:00<00:00, 12.58s/degrees]
ttheta:  80%|████████████████████     | 0.04/0.05 [00:00<00:00, 11.94s/degrees]
ttheta: 100%|█████████████████████████| 0.05/0.05 [00:00<00:00, 11.55s/degrees]
ttheta [In progress. No progress bar available.]                               
                                                                               


In [18]:
uids = RE(th_tth_scan([noisy, th_tth_permit], 8, 6, points=4, min_sep=3))



Transient Scan ID: 5     Time: 2020-09-02 14:41:35
Persistent Unique Scan ID: 'aa1115b8-b722-4f35-b437-69179c43e866'
New stream: 'th_tth_permit_monitor'
Suspending....To get prompt hit Ctrl-C twice to pause.
Suspension occurred at 2020-09-02 14:41:35.
Suspender SuspendBoolLow(EpicsSignal(read_pv='sky:userCalc1.VAL', name='th_tth_permit', value=1.0, timestamp=1599075695.594593, tolerance=1e-05, auto_monitor=False, string=False, write_pv='sky:userCalc1.VAL', limits=False, put_complete=False), sleep=0, pre_plan=None, post_plan=None,tripped_message=) reports a return to nominal conditions. Will sleep for 0 seconds and then release suspension at 2020-09-02 14:41:35.
Justification for this suspension:
Signal th_tth_permit is low


Run aborted
Traceback (most recent call last):
  File "/home/prjemian/Apps/anaconda/envs/bluesky_2020_9/lib/python3.8/site-packages/bluesky/run_engine.py", line 1365, in _run
    msg = self._plan_stack[-1].send(resp)
  File "/home/prjemian/Apps/anaconda/envs/bluesky_2020_9/lib/python3.8/site-packages/bluesky/preprocessors.py", line 1307, in __call__
    return (yield from plan)
  File "/home/prjemian/Apps/anaconda/envs/bluesky_2020_9/lib/python3.8/site-packages/bluesky/preprocessors.py", line 1160, in baseline_wrapper
    return (yield from plan)
  File "/home/prjemian/Apps/anaconda/envs/bluesky_2020_9/lib/python3.8/site-packages/bluesky/preprocessors.py", line 803, in monitor_during_wrapper
    return (yield from plan2)
  File "/home/prjemian/Apps/anaconda/envs/bluesky_2020_9/lib/python3.8/site-packages/bluesky/preprocessors.py", line 170, in plan_mutator
    raise ex
  File "/home/prjemian/Apps/anaconda/envs/bluesky_2020_9/lib/python3.8/site-packages/bluesky/preprocessors.py", line 1

Suspender SuspendBoolLow(EpicsSignal(read_pv='sky:userCalc1.VAL', name='th_tth_permit', value=1.0, timestamp=1599075695.694675, tolerance=1e-05, auto_monitor=False, string=False, write_pv='sky:userCalc1.VAL', limits=False, put_complete=False), sleep=0, pre_plan=None, post_plan=None,tripped_message=) reports a return to nominal conditions. Will sleep for 0 seconds and then release suspension at 2020-09-02 14:41:35.


FailedStatus: MoveStatus(done=True, pos=theta, elapsed=0.2, success=False, settle_time=0.0)

**Failed** during the scan when `theta=3.23` and `ttheta=6.28` and the command line is not responsive.  Move `ttheta` from the GUI in another window.

## Conclusion

We can avoid an anticipated collision of instrument hardware by providing a RunEngine suspender tied to the value of an EPICS PV.